In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import re

In [2]:
df = pd.read_csv(r'C:\Users\SALAH MAHMOUD\Downloads\ieee-mansb-victoris-2\test.csv')

In [3]:
df.head()

,Unnamed: 0,Person Description,Place Code,Customer Order,Additional Features in market,Promotion Name,Store Kind,Store Sales,Store Cost,Gross Weight,Net Weight,Package Weight,Is Recyclable?,Yearly Income,Store Area,Grocery Area,Frozen Area,Meat Area
0,0,"Single Male with four children, education: par...",B06ls_CA,"Candy from Snacks department, Ordered Brand : ...","['Florist', 'Ready Food', 'Video Store', 'Bar ...",Fantastic Discounts,Gourmet,11.76 Millions,4.704 Millions,31.8337,28.7811,3.0525,yes,50K+,2201.06,1424.85,465.54,308.73
1,1,"Married Female with three children, education:...",S01em_OR,"Vegetables from Frozen Foods department, Order...","['Florist', 'Coffee Bar', 'Ready Food', 'Bar F...",Bag Stuffer,Deluxe,2.16 Millions,0.6696 Millions,29.9440,27.0368,2.9072,no,70K+,2577.16,1735.17,505.07,336.59
2,2,"Single Female with three children, education: ...",H11go_ZA,"Dairy from Dairy department, Ordered Brand : Club","['Bar For Salad', 'Video Store', 'Ready Food',...",Pick Your Savings,Deluxe,1.83 Millions,0.8235 Millions,29.2172,26.3100,2.9072,no,130K+,2837.58,2038.11,481.98,321.26
3,3,"Single Male with three children, education: pa...",M10da_YU,"Frozen Desserts from Frozen Foods department, ...","['Coffee Bar', 'Video Store', 'Bar For Salad',...",Price Winners,Deluxe,8.82 Millions,4.41 Millions,28.0543,25.0018,3.0525,no,10K+,2859.04,1871.16,593.93,394.58
4,4,"Single Male with two children, education: high...",L05es_CA,"Hot Beverages from Beverages department, Order...",['Florist'],Dollar Days,Supermarket,4.32 Millions,1.9872 Millions,23.5482,20.6410,2.9072,no,30K+,2193.97,1320.15,523.32,348.85


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19942 entries, 0 to 19941
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     19942 non-null  int64  
 1   Person Description             19942 non-null  object 
 2   Place Code                     19942 non-null  object 
 3   Customer Order                 19942 non-null  object 
 4   Additional Features in market  19942 non-null  object 
 5   Promotion Name                 19916 non-null  object 
 6   Store Kind                     19914 non-null  object 
 7   Store Sales                    19942 non-null  object 
 8   Store Cost                     19942 non-null  object 
 9   Gross Weight                   18959 non-null  float64
 10  Net Weight                     18924 non-null  float64
 11  Package Weight                 19942 non-null  float64
 12  Is Recyclable?                 19942 non-null 

In [5]:
df.rename(columns = {'Unnamed: 0':'ID'}, inplace = True)

In [6]:
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype('category')

In [7]:
# Removing 'Millions' and converting to float for train_1
for col in ['Store Sales', 'Store Cost']:
    df[col] = df[col].astype(str).str.replace('Millions', '').str.replace('illions', '').str.replace('M', '').astype(float) * 1000000


In [8]:
# Function to extract city&country codes
def extract_city_code(place_code):
    parts = place_code.split('_')  # Split by underscore
    if len(parts) == 2:
        return parts[0]
    else:
        return None
# Function to extract city&country codes
def extract_country_code(place_code):
    parts = place_code.split('_')  # Split by underscore
    if len(parts) == 2:
        return parts[1]
    else:
        return None

# Apply the function to create a new 'City Code' column
df['City Code'] = df['Place Code'].apply(extract_city_code)
df['Country Code'] = df['Place Code'].apply(extract_country_code)

In [9]:
df.drop('Place Code',axis=1,inplace = True)

In [10]:
# Define extraction functions
def extract_status(text):
    status_pattern = r'Single|Married'
    status_match = re.search(status_pattern, text)
    return status_match.group() if status_match else None

def extract_children(text):
    children_pattern = r'\b(\w+) children\b'
    children_match = re.search(children_pattern, text)
    return children_match.group(1) if children_match else None

def extract_education(text):
    education_pattern = r'education:\s*(\w+\s*\w+)'
    education_match = re.search(education_pattern, text, re.IGNORECASE)
    return education_match.group(1) if education_match else None

def extract_working(text):
    working_pattern = r'as (\w+(?: \w+)*)'
    working_match = re.search(working_pattern, text, re.IGNORECASE)
    return working_match.group(1) if working_match else None

def extract_sex(text):
    sex_pattern = r'\b(?:Male|Female)\b'
    sex_match = re.search(sex_pattern, text, re.IGNORECASE)
    return sex_match.group() if sex_match else None


# Loop through each DataFrame
df['Status'] = df['Person Description'].apply(extract_status)
df['Number of Children'] = df['Person Description'].apply(extract_children)
df['Education'] = df['Person Description'].apply(extract_education)
df['Working Status'] = df['Person Description'].apply(extract_working)
df['Sex'] = df['Person Description'].apply(extract_sex)

In [11]:
df.drop('Person Description',axis=1,inplace = True)

In [12]:
# Extracts product type from text
def extract_product_type(text):
    pattern = r'^([\w\s]+) from' 
    match = re.search(pattern, text)
    return match.group(1).strip() if match else None

# Extracts ordered brand from text
def extract_ordered_brand(text):
    pattern = r'Ordered Brand : ([\w\s]+)' 
    match = re.search(pattern, text)
    return match.group(1).strip() if match else None

# Extracts department from text
def extract_department(text):
    pattern = r'from ([\w\s]+) department'
    match = re.search(pattern, text)
    return match.group(1).strip() if match else None

# Loop through each DataFrame column
df['Department'] = df['Customer Order'].apply(extract_department) 
df['Ordered_Brand'] = df['Customer Order'].apply(extract_ordered_brand)  
df['Product_Type'] = df['Customer Order'].apply(extract_product_type)  

In [13]:
df.drop('Customer Order',axis=1,inplace = True)

In [14]:
# make new columns from Additional Features in market
def extract_feature(text, feature_name):
    if pd.isna(text):
        return np.nan
    elif feature_name in text:
        return 1
    else:
        return 0

# Define a list of features to extract
features_to_extract = ['Video Store', 'Florist', 'Coffee Bar', 'Bar For Salad', 'Ready Food']

# Loop through the list of features and apply the extraction function
for feature in features_to_extract:
    df[feature] = df['Additional Features in market'].apply(lambda x: extract_feature(x, feature))

In [15]:
df.drop('Additional Features in market',axis=1,inplace = True)

In [16]:
# Mapping of textual values to numerical values
mapping = {
    'No': 0,
    'one': 1,
    'two': 2,
    'three': 3,
    'four': 4,
    'five': 5,
    # Add more mappings as needed
}

# Replace textual values with numerical values
df['Number of Children'] = df['Number of Children'].replace(mapping)

# Convert the column to numeric (this will handle NaN values as well)
df['Number of Children'] = pd.to_numeric(df['Number of Children'], errors='coerce')


In [17]:
df['Yearly Income'] = df['Yearly Income'].str.replace("K", "").str.replace("+", "").str.replace(".", "").astype('float64')


C:\Users\SALAH MAHMOUD\AppData\Local\Temp\ipykernel_2104\1080633508.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Yearly Income'] = df['Yearly Income'].str.replace("K", "").str.replace("+", "").str.replace(".", "").astype('float64')


In [18]:
pd.set_option('display.max_columns', 28)
df.head()

,ID,Promotion Name,Store Kind,Store Sales,Store Cost,Gross Weight,Net Weight,Package Weight,Is Recyclable?,Yearly Income,Store Area,Grocery Area,Frozen Area,Meat Area,...,Country Code,Status,Number of Children,Education,Working Status,Sex,Department,Ordered_Brand,Product_Type,Video Store,Florist,Coffee Bar,Bar For Salad,Ready Food
0,0,Fantastic Discounts,Gourmet,11760000.0,4704000.0,31.8337,28.7811,3.0525,yes,50.0,2201.06,1424.85,465.54,308.73,...,CA,Single,4.0,partial high,skilled manual,Male,Snacks,Gulf Coast,Candy,1,1,1,1,1
1,1,Bag Stuffer,Deluxe,2160000.0,669600.0,29.9440,27.0368,2.9072,no,70.0,2577.16,1735.17,505.07,336.59,...,OR,Married,3.0,graduate degree,professional,Female,Frozen Foods,Golden,Vegetables,1,1,1,1,1
2,2,Pick Your Savings,Deluxe,1830000.0,823500.0,29.2172,26.3100,2.9072,no,130.0,2837.58,2038.11,481.98,321.26,...,ZA,Single,3.0,partial high,management,Female,Dairy,Club,Dairy,1,1,1,1,1
3,3,Price Winners,Deluxe,8820000.0,4410000.0,28.0543,25.0018,3.0525,no,10.0,2859.04,1871.16,593.93,394.58,...,YU,Single,3.0,partial high,manual,Male,Frozen Foods,Imagine,Frozen Desserts,1,1,1,1,1
4,4,Dollar Days,Supermarket,4320000.0,1987200.0,23.5482,20.6410,2.9072,no,30.0,2193.97,1320.15,523.32,348.85,...,CA,Single,2.0,high school,manual,Male,Beverages,BBB Best,Hot Beverages,0,1,0,0,0


In [19]:
# Define a function 'Income' that categorizes income values
def Income(x):
    if x <= 70:
        return 'less_70'  # If income is less than or equal to 70, categorize as 0
    elif x > 70:
        return 'greater_70'  # If income is greater than 70, categorize as 1

df['Yearly Income'] = df['Yearly Income'].apply(Income)

In [20]:
def Is_Recyclable(x):
    if x == 'recyclable':
        return 'yes'
    elif x == 'non recyclable':  
        return 'no'
    elif x == 'yes':
        return 'yes'
    elif x == 'no':return 'no'

df['Is Recyclable?'] = df['Is Recyclable?'].apply(Is_Recyclable)

In [21]:
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype('category')

In [22]:
pd.set_option('display.max_columns', 28)
df.head()

,ID,Promotion Name,Store Kind,Store Sales,Store Cost,Gross Weight,Net Weight,Package Weight,Is Recyclable?,Yearly Income,Store Area,Grocery Area,Frozen Area,Meat Area,...,Country Code,Status,Number of Children,Education,Working Status,Sex,Department,Ordered_Brand,Product_Type,Video Store,Florist,Coffee Bar,Bar For Salad,Ready Food
0,0,Fantastic Discounts,Gourmet,11760000.0,4704000.0,31.8337,28.7811,3.0525,yes,less_70,2201.06,1424.85,465.54,308.73,...,CA,Single,4.0,partial high,skilled manual,Male,Snacks,Gulf Coast,Candy,1,1,1,1,1
1,1,Bag Stuffer,Deluxe,2160000.0,669600.0,29.9440,27.0368,2.9072,no,less_70,2577.16,1735.17,505.07,336.59,...,OR,Married,3.0,graduate degree,professional,Female,Frozen Foods,Golden,Vegetables,1,1,1,1,1
2,2,Pick Your Savings,Deluxe,1830000.0,823500.0,29.2172,26.3100,2.9072,no,greater_70,2837.58,2038.11,481.98,321.26,...,ZA,Single,3.0,partial high,management,Female,Dairy,Club,Dairy,1,1,1,1,1
3,3,Price Winners,Deluxe,8820000.0,4410000.0,28.0543,25.0018,3.0525,no,less_70,2859.04,1871.16,593.93,394.58,...,YU,Single,3.0,partial high,manual,Male,Frozen Foods,Imagine,Frozen Desserts,1,1,1,1,1
4,4,Dollar Days,Supermarket,4320000.0,1987200.0,23.5482,20.6410,2.9072,no,less_70,2193.97,1320.15,523.32,348.85,...,CA,Single,2.0,high school,manual,Male,Beverages,BBB Best,Hot Beverages,0,1,0,0,0


In [23]:
expected_column_order = ['ID','Promotion Name', 'Store Kind', 'Store Sales', 'Is Recyclable?',
       'Yearly Income', 'Store Area', 'Meat Area', 'Gross Weight',
       'Package Weight', 'City Code', 'Country Code', 'Status',
       'Number of Children', 'Working Status', 'Sex', 'Department',
       'Ordered_Brand', 'Product_Type', 'Video Store', 'Florist', 'Coffee Bar',
       'Ready Food']

df = df[expected_column_order]

In [24]:
X_test = df.drop(['ID'],axis = 1)

In [25]:
# Define columns by data type
numeric_cols = [col for col in X_test.select_dtypes(include=['float64']).columns]
categorical_cols = [col for col in X_test.select_dtypes(include=['category']).columns]

In [29]:
import joblib

In [30]:
model = joblib.load('final_model.joblib')

In [31]:
# Assuming predictions is a multi-dimensional array
predictions = model.predict(X_test)

# Create submission DataFrame
submission_df = pd.DataFrame({
    "ID": df["ID"].values,
    "Cost": predictions
})

# Save submission as CSV
submission_df.to_csv("final_test.csv", index=False)
